In [1]:
import numpy as np
import cv2
import glob

In [2]:
patch_x = 7
patch_y = 7
patch_size = (7  ,7)

In [3]:
# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

In [4]:
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((patch_x * patch_y, 3), np.float32)
objp[:,:2] = np.mgrid[0:patch_y , 0:patch_x].T.reshape(-1,2)

In [5]:
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

In [6]:
images = glob.glob('calib/*.jpg')

In [8]:
images

['calib\\IMG_5933.JPG',
 'calib\\IMG_5934.JPG',
 'calib\\IMG_5935.JPG',
 'calib\\IMG_5936.JPG',
 'calib\\IMG_5937.JPG',
 'calib\\IMG_5938.JPG']

In [18]:
#img = cv2.imread(images[1])

In [18]:
#cv2.imwrite('corner\\img.jpg',img)

In [11]:
#gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

In [12]:
#ret, corners = cv2.findChessboardCorners(gray,
#                                         patch_size,
#                                         flags=cv2.CALIB_CB_ADAPTIVE_THRESH
#                                             +cv2.CALIB_CB_NORMALIZE_IMAGE
#                                             +cv2.CALIB_CB_FAST_CHECK)

In [19]:
#ret

In [12]:
#cv2.imshow('img',img)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

In [13]:
#cv2.imshow('gray',gray)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

## Find Corners

In [9]:
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
    print(fname)

    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray,
                                         patch_size,
                                         flags=cv2.CALIB_CB_ADAPTIVE_THRESH
                                             +cv2.CALIB_CB_NORMALIZE_IMAGE
                                             +cv2.CALIB_CB_FAST_CHECK)
    
    print(ret)

    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)

        corners2 = cv2.cornerSubPix(gray, corners,(11,11),(-1,-1), criteria)
        imgpoints.append(corners2)

        # Draw and display the corners
        cornerImage = cv2.drawChessboardCorners(img, patch_size, corners2, ret)
        cv2.imwrite('corner\\' + fname[6:], cornerImage)

calib\IMG_5933.JPG
True
calib\IMG_5934.JPG
True
calib\IMG_5935.JPG
True
calib\IMG_5936.JPG
True
calib\IMG_5937.JPG
True
calib\IMG_5938.JPG
True


## Calibration

In [10]:
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)

In [11]:
ret

1.8516187758164635

In [12]:
mtx

array([[  2.79090673e+03,   0.00000000e+00,   1.31897282e+03],
       [  0.00000000e+00,   2.82053910e+03,   1.55192556e+03],
       [  0.00000000e+00,   0.00000000e+00,   1.00000000e+00]])

In [13]:
dist

array([[-0.03647438,  0.28956809, -0.01643535,  0.00494458, -0.10907848]])

In [14]:
rvecs

[array([[ 0.33167397],
        [ 0.02283066],
        [ 0.00641129]]), array([[ 0.25408547],
        [-0.37276938],
        [ 1.52517202]]), array([[ 0.19152467],
        [-0.50160023],
        [ 1.51292302]]), array([[ 0.33292191],
        [ 0.17091289],
        [ 0.042596  ]]), array([[ 0.16379205],
        [-0.08290412],
        [ 0.00913219]]), array([[ 0.46675176],
        [-0.01624671],
        [ 0.0147052 ]])]

In [15]:
tvecs

[array([[ -3.17022522],
        [ -2.47431489],
        [ 10.78877207]]), array([[  2.59587117],
        [ -1.64277942],
        [ 11.2099658 ]]), array([[  2.75511232],
        [ -1.43347073],
        [ 11.46829099]]), array([[ -3.58104461],
        [ -2.44624969],
        [ 10.60997451]]), array([[ -3.28759361],
        [ -2.49083866],
        [ 10.61334297]]), array([[ -3.50668294],
        [ -2.06214617],
        [ 11.56194667]])]

## Undistortion

1 Using cv2.undistort()

In [16]:
targetimages = glob.glob('source/*.jpg')
for Iname in targetimages:
    img = cv2.imread(Iname)
    h,  w = img.shape[:2]
    newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist,(w,h), 1,(w,h))

    # undistort
    dst = cv2.undistort(img, mtx, dist, None, newcameramtx)

    # crop the image
    x,y,w,h = roi
    dst = dst[y:y+h, x:x+w]
    cv2.imwrite('undis1\\' + Iname[7:],dst)

2 Using remapping

In [17]:
targetimages = glob.glob('source/*.jpg')
for Iname in targetimages:
    img = cv2.imread(Iname)
    h,  w = img.shape[:2]
    newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))

    # undistort
    mapx,mapy = cv2.initUndistortRectifyMap(mtx,dist,None,newcameramtx,(w,h),5)
    dst = cv2.remap(img,mapx,mapy,cv2.INTER_LINEAR)

    # crop the image
    x,y,w,h = roi
    dst = dst[y:y+h, x:x+w]
    cv2.imwrite('undis2\\' + Iname[7:],dst)